# Evidence Detection

## Data is in the form of: Claim, Evidence, Labels

## Labels
- **1 (Relevant)** - The evidence supports or is related to the claim.
- **0 (Not Relevant)** – The evidence does not support or is unrelated to the claim.

### Dependency Management

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 19.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
import pickle
import os
import optuna

# Create a directory for NLTK data
nltk_data_dir = os.path.join(os.getcwd(), 'nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)

# Set the NLTK data path
nltk.data.path.append(nltk_data_dir)

# Download punkt to the specified directory
nltk.download('punkt_tab', download_dir=nltk_data_dir)

[nltk_data] Downloading package punkt_tab to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
from evidence_detection.bilstm_with_attention import BiLSTMAttention
from evidence_detection.evidence_detection_dataset import EvidenceDetectionDataset
from evidence_detection.vocabulary import Vocabulary
from evidence_detection.trainer import Trainer

In [5]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create output directory for models and plots
os.makedirs('models', exist_ok=True)
# os.makedirs('plots', exist_ok=True)

Using device: cuda


### Define Glove Embedding Method

In [6]:
def load_glove_embeddings(vocab, glove_path, embedding_dim=300):
    """Load GloVe embeddings for words in vocabulary"""
    embeddings = np.zeros((len(vocab), embedding_dim))

    # Initialize random embeddings
    for i in range(len(vocab)):
        embeddings[i] = np.random.normal(scale=0.1, size=(embedding_dim, ))

    # Load pretrained embeddings
    if not os.path.exists(glove_path):
        print(f"GloVe embeddings not found at {glove_path}. Using random embeddings.")
        return embeddings

    print(f"Loading GloVe embeddings from {glove_path}...")

    with open(glove_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(tqdm(f, desc="Loading GloVe")):
            try:
                values = line.split()

                # Check if the vector has the correct dimension
                if len(values) != embedding_dim + 1:  # +1 for the word itself
                    print(f"Warning: Line {i} has {len(values)} values, expected {embedding_dim + 1}. Skipping.")
                    continue

                word = values[0]
                if word in vocab.stoi:
                    vector = np.array(values[1:], dtype='float32')

                    # Double-check vector dimension
                    if len(vector) != embedding_dim:
                        print(f"Warning: Vector for word '{word}' has dimension {len(vector)}, expected {embedding_dim}. Skipping.")
                        continue

                    embeddings[vocab.stoi[word]] = vector
            except Exception as e:
                print(f"Error processing line {i}: {e}")
                continue

    print(f"Loaded {embedding_dim}-dimensional GloVe embeddings.")
    return embeddings

### Get Data, build vocabulary and Build Glove Embeddings

In [7]:
# Load datasets
print("Loading datasets...")
# train_df = pd.read_csv('./data/train.csv')
# val_df = pd.read_csv('./data/dev.csv')
print("Using train.csv for train and validation")
print("Using dev.csv for testing")
train_df = pd.read_csv('./data/train.csv')
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
test_df = pd.read_csv('./data/dev.csv')

print(f"Train shape: {train_df.shape}, Validation shape: {val_df.shape}")
print(f"Label distribution in train: {train_df['label'].value_counts().to_dict()}")
print(f"Label distribution in val: {val_df['label'].value_counts().to_dict()}")

Loading datasets...
Using train.csv for train and validation
Using dev.csv for testing
Train shape: (17206, 3), Validation shape: (4302, 3)
Label distribution in train: {0: 12504, 1: 4702}
Label distribution in val: {0: 3150, 1: 1152}


In [8]:
# Load or create vocabulary
vocab_path = 'vocab.pkl'
if os.path.exists(vocab_path):
    print(f"Loading vocabulary from {vocab_path}")
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)
else:
    print("Creating new vocabulary")
    vocab = Vocabulary(freq_threshold=3)
    all_texts = train_df['Claim'].tolist() + train_df['Evidence'].tolist()
    vocab.build_vocabulary(all_texts)

    # Save vocabulary
    with open(vocab_path, 'wb') as f:
        pickle.dump(vocab, f)

print(f"Vocabulary size: {len(vocab)}")

Creating new vocabulary


Building vocabulary: 100%|██████████| 34412/34412 [00:03<00:00, 8897.85it/s]

Vocabulary size: 12686
Vocabulary size: 12686


In [9]:
# Load GloVe embeddings
embedding_dim = 300
glove_path = "glove.6B.300d.txt"
embeddings = load_glove_embeddings(vocab, glove_path, embedding_dim)

Loading GloVe embeddings from glove.6B.300d.txt...


Loading GloVe: 104968it [00:02, 44579.87it/s]

Loaded 300-dimensional GloVe embeddings.


### Hyperparameter Tuning

In [38]:
# Create datasets
train_dataset = EvidenceDetectionDataset(train_df, vocab)
val_dataset = EvidenceDetectionDataset(val_df, vocab)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

num_layers = 1
# Define objective function for Optuna
def objective(trial):
    # Hyperparameters to optimize
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    # num_layers = trial.suggest_int('num_layers', [1])
    dropout = trial.suggest_float('dropout', 0.3, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-4, log=True)

    # Initialize model with suggested parameters
    model = BiLSTMAttention(
        vocab_size=len(vocab),
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout,
        pretrained_embeddings=embeddings
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        device=device
    )

    # Train for a few epochs
    trainer.train(num_epochs=5)  # Reduced epochs for faster hyperparameter search

    # Evaluate on validation set
    val_metrics = trainer.evaluate()

    # Return F1 score as the optimization metric
    return val_metrics['f1']

# Create study directory
os.makedirs('optuna_results', exist_ok=True)

# Create and run Optuna study
study = optuna.create_study(direction='maximize',
                           study_name='bilstm_attention_optimization'
                          #  storage='sqlite:///optuna_results/bilstm_attention.db'
                            )
study.optimize(objective, n_trials=20)  # Adjust number of trials as needed

# Print and save best parameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (F1 Score): {trial.value:.4f}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Save best parameters
with open('optuna_results/best_params.pkl', 'wb') as f:
    pickle.dump(trial.params, f)

[I 2025-03-21 17:29:54,973] A new study created in memory with name: bilstm_attention_optimization


Train dataset size: 17206
Validation dataset size: 4302
Starting training on device: cuda
Training set size: 17206
Validation set size: 4302


Validating: 100%|██████████| 269/269 [00:02<00:00, 109.64it/s]


Epoch 1/5 - Time: 25.40s
Train Loss: 0.4384, Train F1: 0.5560
Val Loss: 0.3910, Val F1: 0.5648
Val Precision: 0.7746, Val Recall: 0.4444
Validation F1 increased (0.000000 --> 0.564810). Saving model...


Validating: 100%|██████████| 269/269 [00:02<00:00, 109.60it/s]


...

Epoch 4/5 - Time: 24.54s
Train Loss: 0.2646, Train F1: 0.7894
Val Loss: 0.4027, Val F1: 0.6623
Val Precision: 0.6589, Val Recall: 0.6658
EarlyStopping counter: 1 out of 3


Validating: 100%|██████████| 269/269 [00:02<00:00, 108.69it/s]


Epoch 5/5 - Time: 24.57s
Train Loss: 0.2048, Train F1: 0.8536
Val Loss: 0.4366, Val F1: 0.6675
Val Precision: 0.6441, Val Recall: 0.6927
EarlyStopping counter: 2 out of 3
Loaded best model from 'best_bilstm_attention_model.pt'


Validating: 100%|██████████| 269/269 [00:02<00:00, 109.79it/s]
[I 2025-03-21 18:01:23,047] Trial 19 finished with value: 0.6794092093831451 and parameters: {'hidden_dim': 128, 'dropout': 0.3864588188567415, 'batch_size': 16, 'learning_rate': 0.0002398422178544341, 'weight_decay': 1.5113095854152683e-05}. Best is trial 18 with value: 0.6911290322580645.


Best trial:
  Value (F1 Score): 0.6911
  Params: 
    hidden_dim: 512
    dropout: 0.4508594141808036
    batch_size: 16
    learning_rate: 0.00027416000685940727
    weight_decay: 9.847501948528487e-05


### Train Model

In [39]:
# Train final model with best hyperparameters
print("\nTraining final model with best hyperparameters...")
model = BiLSTMAttention(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    hidden_dim=trial.params['hidden_dim'],
    # num_layers=trial.params['num_layers'],
    num_layers=num_layers,
    dropout=trial.params['dropout'],
    pretrained_embeddings=embeddings
)


# Print model architecture summary
print("\nModel Architecture:")
print(f"Vocabulary Size: {len(vocab)}")
print(f"Embedding Dimension: {embedding_dim}")
print(f"Hidden Dimension: {trial.params['hidden_dim']}")
# print(f"Batch size: {trial.params['batch_size']}")
print(f"Number of LSTM Layers: {num_layers}")
print(f"Dropout Rate: {trial.params['dropout']}")



# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

# Initialize trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=trial.params['batch_size'],
    learning_rate=trial.params['learning_rate'],
    weight_decay=trial.params['weight_decay'],
    device=device
)

# Train model
print("\nStarting training...")
trainer.train(num_epochs=15)

# Final evaluation
print("\nPerforming final evaluation on validation set...")
val_metrics = trainer.evaluate()

print("\nFinal Validation Metrics:")
print(f"Loss: {val_metrics['loss']:.4f}")
print(f"Accuracy: {val_metrics['accuracy']:.4f}")
print(f"Precision: {val_metrics['precision']:.4f}")
print(f"Recall: {val_metrics['recall']:.4f}")
print(f"F1 Score: {val_metrics['f1']:.4f}")

# Save final model
final_model_path = 'models/final_model.pt'
trainer.save_model(final_model_path)

print("\nTraining and evaluation completed!")


Training final model with best hyperparameters...

Model Architecture:
Vocabulary Size: 12686
Embedding Dimension: 300
Hidden Dimension: 512
Number of LSTM Layers: 1
Dropout Rate: 0.4508594141808036
Total Parameters: 6,880,364
Trainable Parameters: 6,880,364

Starting training...
Starting training on device: cuda
Training set size: 17206
Validation set size: 4302


Validating: 100%|██████████| 269/269 [00:02<00:00, 109.06it/s]


Epoch 1/15 - Time: 25.80s
Train Loss: 0.4565, Train F1: 0.5149
Val Loss: 0.3946, Val F1: 0.5839
Val Precision: 0.7453, Val Recall: 0.4800
Validation F1 increased (0.000000 --> 0.583949). Saving model...


Validating: 100%|██████████| 269/269 [00:02<00:00, 108.51it/s]


Epoch 2/15 - Time: 25.80s
Train Loss: 0.3796, Train F1: 0.6497
Val Loss: 0.3715, Val F1: 0.6736
Val Precision: 0.6772, Val Recall: 0.6701
Validation F1 increased (0.583949 --> 0.673647). Saving model...


...

Epoch 4/15 - Time: 25.84s
Train Loss: 0.2974, Train F1: 0.7559
Val Loss: 0.3733, Val F1: 0.6628
Val Precision: 0.6751, Val Recall: 0.6510
EarlyStopping counter: 2 out of 3


Validating: 100%|██████████| 269/269 [00:02<00:00, 108.83it/s]


Epoch 5/15 - Time: 25.47s
Train Loss: 0.2458, Train F1: 0.8075
Val Loss: 0.4039, Val F1: 0.6718
Val Precision: 0.6650, Val Recall: 0.6788
EarlyStopping counter: 3 out of 3
Early stopping triggered
Loaded best model from 'best_bilstm_attention_model.pt'

Performing final evaluation on validation set...


Validating: 100%|██████████| 269/269 [00:02<00:00, 109.35it/s]


Final Validation Metrics:
Loss: 0.3715
Accuracy: 0.8261
Precision: 0.6772
Recall: 0.6701
F1 Score: 0.6736
Model saved to models/final_model.pt

Training and evaluation completed!


In [40]:
# Create the dataset
test_dataset = EvidenceDetectionDataset(test_df, vocab)
print(f"Test dataset size: {len(test_dataset)}")

# Create test data loader for final testing
test_loader = DataLoader(
    test_dataset,
    batch_size=trial.params['batch_size'],
    shuffle=False,
    collate_fn=trainer.collate_fn
)

# Load the final model
final_model_path = 'models/final_model.pt'
final_model = BiLSTMAttention(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    hidden_dim=trial.params['hidden_dim'],
    num_layers=num_layers,
    dropout=trial.params['dropout'],
    pretrained_embeddings=embeddings
)
final_model.load_state_dict(torch.load(final_model_path))
final_model.to(device)

# Now evaluate on test set
print("\nPerforming evaluation on test set (dev.csv)...")
final_model.eval()
test_loss = 0.0
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        # Move batch to device
        claim_ids = batch['claim_ids'].to(device)
        claim_lengths = batch['claim_lengths']
        evidence_ids = batch['evidence_ids'].to(device)
        evidence_lengths = batch['evidence_lengths']
        labels = batch['labels'].to(device)

        # Forward pass
        logits, _ = final_model(claim_ids, claim_lengths, evidence_ids, evidence_lengths)
        loss = trainer.criterion(logits, labels)

        # Accumulate metrics
        test_loss += loss.item()
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        all_predictions.extend(predictions)
        all_labels.extend(labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='binary')
recall = recall_score(all_labels, all_predictions, average='binary')
f1 = f1_score(all_labels, all_predictions, average='binary')

print("\nTest Set Metrics:")
print(f"Loss: {test_loss / len(test_loader):.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test dataset size: 5926

Performing evaluation on test set (dev.csv)...


Testing: 100%|██████████| 371/371 [00:03<00:00, 109.15it/s]


Test Set Metrics:
Loss: 0.3840
Accuracy: 0.8134
Precision: 0.6716
Recall: 0.6372
F1 Score: 0.6539


### Test the model

In [41]:
# Function to collate batches without labels for test data
def test_collate_fn(batch):
    # Separate batch elements
    claims = [item['claim_ids'] for item in batch]
    claim_lengths = torch.tensor([item['claim_length'] for item in batch])
    evidences = [item['evidence_ids'] for item in batch]
    evidence_lengths = torch.tensor([item['evidence_length'] for item in batch])

    # Pad sequences
    padded_claims = pad_sequence(claims, batch_first=True, padding_value=0)
    padded_evidences = pad_sequence(evidences, batch_first=True, padding_value=0)

    return {
        'claim_ids': padded_claims,
        'claim_lengths': claim_lengths,
        'evidence_ids': padded_evidences,
        'evidence_lengths': evidence_lengths
    }

# Set device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device: {device}")

# Load vocabulary
# with open('vocab.pkl', 'rb') as f:
#     vocab = pickle.load(f)
# print(f"Vocabulary size: {len(vocab)}")

# Load the model with the same hyperparameters
model = BiLSTMAttention(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    hidden_dim=trial.params['hidden_dim'],
    num_layers=num_layers,
    dropout=trial.params['dropout'],
    pretrained_embeddings=embeddings
)

# Load trained model weights
model.load_state_dict(torch.load('models/final_model.pt', map_location=device))
model = model.to(device)
model.eval()
print("Model loaded successfully")

# Load test data
# test_file = './data/dev.csv'  # Change to your test file path
test_df = pd.read_csv('./data/dev.csv')
print(f"Test data loaded: {len(test_df)} rows")

# Create test dataset
test_dataset = EvidenceDetectionDataset(test_df, vocab, is_test=True)
print(f"Test dataset created: {len(test_dataset)} samples")

# Create test dataloader
batch_size = 32
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=test_collate_fn
)

# Run inference
print("Running inference on test data...")
all_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        # Move batch to device
        claim_ids = batch['claim_ids'].to(device)
        claim_lengths = batch['claim_lengths']
        evidence_ids = batch['evidence_ids'].to(device)
        evidence_lengths = batch['evidence_lengths']

        # Forward pass
        logits, _ = model(claim_ids, claim_lengths, evidence_ids, evidence_lengths)

        # Get predictions (0 or 1)
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        all_predictions.extend(predictions)

# Save predictions to CSV
predictions_df = pd.DataFrame({'prediction': all_predictions})
predictions_df.to_csv('predictions.csv', index=False)
print(f"Predictions saved to 'predictions.csv'")

Model loaded successfully
Test data loaded: 5926 rows
Test dataset created: 5926 samples
Running inference on test data...


Testing: 100%|██████████| 186/186 [00:02<00:00, 73.30it/s]

Predictions saved to 'predictions.csv'
